First, we manage all imports for the code.

In [ ]:
!pip install gensim

In [15]:
#imports
import pandas as pd
import numpy as np
import random
import ast
from torch import nn
import torch
import re
import nltk
from torch.utils.data import random_split, Subset, DataLoader
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import zipfile
import os
import shutil

Next we process the zipped data to csv. It is stored in .zip, because it is smaller on git and then locally it is extracted to csv.

In [2]:
with zipfile.ZipFile("data/dataset.zip", "r") as zip_ref:
    zip_ref.extractall("data")

for file_name in os.listdir("data/dataset"):
    source = "data/dataset/" + file_name
    destination = "data/" + file_name
    if os.path.isfile(source): shutil.move(source, destination)

os.rmdir("data/dataset")
del source, destination

Preprocessing of the datasets, the goal is to receive a table with the following columns: Name, Lyrics, Genre

In [3]:

df = pd.read_csv('data/Spotify-2000.csv')
df = df[['Title', 'Top Genre']] #take only the name and genre
df2 = pd.read_csv('data/spotify_millsongdata.csv')
df['lyrics'] = ' ' # add column lyrics
#Now combine those two documents by the title
found = 0

for x, title in enumerate(df['Title']):
    #print(title)
    for y, title2 in enumerate(df2['song']):
        if title2.lower() == title.lower():
            df['lyrics'].iloc[x] = df2['text'].iloc[y]

print("finished combining..")

finished combining..


In [16]:
#remove songs which were not in both datsets
songs_to_remove = []
for x, lyrics in enumerate(df['lyrics']):
    if lyrics == '':
        songs_to_remove.append(x)
df.drop(songs_to_remove, axis = 0, inplace = True)

print(df)

                  Title          Top Genre  \
0               Sunrise    adult standards   
1           Black Night         album rock   
3         The Pretender  alternative metal   
6     She Will Be Loved                pop   
8        Mr. Brightside        modern rock   
...                 ...                ...   
1988         Summertime    adult standards   
1989   Heartbreak Hotel    adult standards   
1990          Hound Dog    adult standards   
1991    Johnny B. Goode         blues rock   
1993     Blueberry Hill    adult standards   

                                                 lyrics  
0     You take away the breath I was keeping for sun...  
1     Black night is not right,  \r\nI don't feel so...  
3     I'm going to rent myself a house  \r\nIn the s...  
6     Beauty queen of only eighteen she  \r\nHad som...  
8     Coming out of my cage  \r\nAnd I've been doing...  
...                                                 ...  
1988  Summertime....and the livin' is eas

Further preprocessing of the lyrics itself. We remove the stopwords and punctuations with regex and stopwords from ntlk

In [11]:
from nltk.corpus import stopwords
nltk.download('stopwords')

sw = stopwords.words('english')
punc_regex = r'[^\s\w]' #searches for everything thats not a word or space
stopword_regex = r'\b{0}\b'

print(df['lyrics'].iloc[1])

for x, lyrics in enumerate(df['lyrics']):
    txt = lyrics
    txt = re.sub(punc_regex, '', txt) #remove punuctuations
    for sword in sw:
        txt = re.sub(stopword_regex.format(sword), '', txt, flags=re.IGNORECASE) #remove every stopword
    df['lyrics'].iloc[x] = txt

#print(df)
print(df['lyrics'].iloc[1])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\k-mar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Black night   right  
 dont feel  bright  
 dont care  sit tight  
Maybe Ill find   way   line  
 Im free free     
Black night   long way  home  
  
 dont need  dark tree  
 dont want  rough sea  
 cant feel  cant see  
Maybe Ill find   way   line  
 Im free free     
Black night   long way  home  
  
Black night black night  
 dont need black night  
 cant see dark night  
Maybe Ill find   way   line  
 Im free free     
Black night   long way  home  


Black night   right  
 dont feel  bright  
 dont care  sit tight  
Maybe Ill find   way   line  
 Im free free     
Black night   long way  home  
  
 dont need  dark tree  
 dont want  rough sea  
 cant feel  cant see  
Maybe Ill find   way   line  
 Im free free     
Black night   long way  home  
  
Black night black night  
 dont need black night  
 cant see dark night  
Maybe Ill find   way   line  
 Im free free     
Black night   long way  home  




Tokenize the lyrics and create tagged Documents

In [52]:
tagged_data = []

for i,d in enumerate(df['lyrics']):
    tokenized_words = nltk.tokenize.word_tokenize(d)
    tagged_data.append(TaggedDocument(words=tokenized_words, tags=str(i)))

print(tagged_data)

[TaggedDocument(words=['take', 'away', 'breath', 'keeping', 'sunrise', 'appear', 'morning', 'looks', 'drab', 'eyes', 'Ill', 'turn', 'love', 'seen', 'youll', 'disappear', 'morning', 'put', 'shame', 'take', 'away', 'breath', 'keeping', 'sunrise', 'appear', 'morning', 'looks', 'drab', 'eyes', 'Ill', 'turn', 'love', 'seen', 'youll', 'disappear', 'morning', 'put', 'shame', 'Sometimes', 'fear', 'go', 'life', 'day', 'spend', 'echoed', 'vision', 'Ill', 'turn', 'love', 'Remembering', 'smile', 'every', 'day', 'spent', 'Thinking', 'times', 'Ive', 'let', 'heads', 'spinning', 'round', 'eyes', 'see', 'chances', 'Ive', 'lost', 'Opportunities', 'tossed', 'Away', 'blue', 'take', 'away', 'breath', 'keeping', 'sunrise', 'appear', 'morning', 'looks', 'drab', 'eyes', 'Ill', 'turn', 'love', 'seen', 'youll', 'disappear', 'morning', 'put', 'shame'], tags='0'), TaggedDocument(words=['Black', 'night', 'right', 'dont', 'feel', 'bright', 'dont', 'care', 'sit', 'tight', 'Maybe', 'Ill', 'find', 'way', 'line', 'Im',

Setting up the Doc2Vec model

In [63]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=1, epochs=30)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

print("Doc2Vec model finished training")

Doc2Vec model finished training


Building the DataLoader for the machine learning model

In [73]:
columns = ['Token'] + ['Target']
df_for_dataloader = pd.DataFrame(columns = columns)
df_for_dataloader.set_index(columns)

for i in range(len(tagged_data)):
    token = [model.infer_vector(tagged_data[i][0])]
    #token = [list(model.dv[i])]
    target = [5] # should be genre
    row = pd.DataFrame([token + target], columns=['Token', 'Target'])
    df_for_dataloader = pd.concat([df_for_dataloader, row])
    
display(df_for_dataloader.iloc[:5]) 

,Token,Target
0,"[-1.4568746, 0.14817706, -0.59844536, -0.23989...",5
0,"[-0.40880728, 0.04540625, -0.51874304, -0.1025...",5
0,"[6.557465e-05, -0.0030068492, -0.0011718646, 0...",5
0,"[0.17799388, 1.650082, 0.17131579, 0.7530677, ...",5
0,"[-0.0024998381, -0.001913867, -0.0032721334, 0...",5


In [37]:
device = torch.device('cpu') #torch.device('cuda' if torch.cuda.is_available() else 'cpu') For faster training in the end

class SongModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(SongModel, self).__init__()
        self.input_size = input_size
        self.hidden_size = input_size * 3
        self.output_size = output_size
        self.layer_1 =   nn.Linear(self.input_size, self.hidden_size)
        self.layer_2 =   nn.Linear(self.hidden_size, self.hidden_size)
        self.layer_out = nn.Linear(self.hidden_size, self.output_size)
        self.softmax =   nn.Softmax()
        self.sigmoid =   nn.Sigmoid()
        self.tanh =      nn.Tanh()

        nn.init.xavier_uniform_(self.layer_1.weight)
        nn.init.zeros_(self.layer_1.bias)
        nn.init.xavier_uniform_(self.layer_2.weight)
        nn.init.zeros_(self.layer_2.bias)
        nn.init.xavier_uniform_(self.layer_out.weight)
        nn.init.zeros_(self.layer_out.bias)


    def forward(self, d):
        x = self.sigmoid(self.layer_1(d))
        x = self.tanh(self.layer_2(x))
        x = self.softmax(self.layer_out(x))
        return x

model = SongModel(1, 3).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.025)
loss_fn = torch.nn.CrossEntropyLoss()

In [38]:
def train(dataloader, model, loss_fn, optimizer):
    total_loss = 0
    iterations = 0
    for sample in dataloader:
        model_input = sample[0]
        should = sample[1]
        predict = model(model_input)
        loss = loss_fn(predict, should.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss
        iterations += 1
        if iterations % 1000 == 0:
            print(f"Predict: {predict}, Loss: {loss}")

    print(total_loss / len(dataloader))
    return